# F1 Fantasy Program

## ReadMe

The goal of this program is to webscrape, and gather, F1 data. The data would then be saved, transformed, and analysed.

## Setup

In [261]:
from datetime import datetime
from bs4 import BeautifulSoup
import pandas as pd
import requests
import glob

import fastf1

### Enable Caching

In [251]:
fastf1.Cache.enable_cache('cache')

### Ignore Warnings

In [252]:
import warnings
warnings.filterwarnings('ignore', category=FutureWarning)

## Data Aquisition

### Get the completed events of the season (year)

In [253]:
def get_completed_events(season, b_print=False):
    schedule = fastf1.get_event_schedule(season)
    # print(schedule)
    completed_events = []
    for event in schedule['RoundNumber'][1:]:
        if schedule['EventDate'][event] < datetime.now():
            completed_events.append(schedule.iloc[[event]])        
    return completed_events

### Load the data of a completed session

In [254]:
def get_session_data(season, event_num, session_type):
    df_session = fastf1.get_session(season, event_num, session_type)
    df_session.load()
    return df_session

### Load the data of all the completed sessions

In [255]:
# def get_completed_sessions(season, completed_events, session_type):
#     completed_sessions = []
#     for i, event in enumerate(completed_events):
#         type(completed_events)
#         session = get_session_data(season, event['RoundNumber'], session_type)
#         completed_sessions.append(session)
#     return completed_sessions

In [256]:
def get_completed_sessions(season, completed_events, session_type):
    # Returns a list of dataframes
    completed_sessions = []

    for event_df in completed_events:
        for _, event in event_df.iterrows():
            session = get_session_data(season, event['RoundNumber'], session_type)
            completed_sessions.append(session)

    return completed_sessions


## Save sessions to csv

In [259]:
def save_df_list_to_csv(dataframe_list, name="dataframe"):
    for i, dataframe in enumerate(dataframe_list):
        file_name = name + '_' + str(i) + '.csv'
        dataframe.to_csv(file_name)

## TEST

In [266]:
season = 2023
event_num = 1
session_type = 'R'

# session_data = get_session_data(season, event_num, session_type)
# print(session_data)

completed_events = get_completed_events(season)
# print(completed_events)
completed_sessions = get_completed_sessions(season, completed_events, session_type) 
print(completed_sessions[1])

core           INFO 	Loading data for Bahrain Grand Prix - Race [v2.3.1]
api            INFO 	Using cached data for driver_info
api            INFO 	Using cached data for timing_data
api            INFO 	Using cached data for timing_app_data
core           INFO 	Processing timing data...
api            INFO 	Using cached data for session_status_data
api            INFO 	Using cached data for track_status_data
api            INFO 	Using cached data for car_data
api            INFO 	Using cached data for position_data
api            INFO 	Using cached data for weather_data
api            INFO 	Using cached data for race_control_messages
core           INFO 	Finished loading data for 20 drivers: ['1', '11', '14', '55', '44', '18', '63', '77', '10', '23', '22', '2', '20', '21', '27', '24', '4', '31', '16', '81']
core           INFO 	Loading data for Saudi Arabian Grand Prix - Race [v2.3.1]
api            INFO 	Using cached data for driver_info
api            INFO 	Using cached data for tim

In [271]:
results_list = [session.results for session in completed_sessions]
save_df_list_to_csv(results_list)